In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our utilities functions
import utils

## Prepare the Data

In [2]:
# Get the original DataFrame
df = utils.get_data()
df.head()

,user_id,movie_id,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [3]:
# Get the normalized ratings
normalized_ratings = utils.get_normalized_data()
normalized_ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.4,0.8,0.7,0.6,0.8,0.7,0.7,0.6,0.5,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.6,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Get the weights and bias tensors
W = utils.weights()
hb = utils.hidden_bias()
vb = utils.visible_bias()

In [5]:
# Get the users so I can send the users back into the model
users = normalized_ratings.index
users

Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508],
      dtype='int64', name='user_id', length=1508)

In [6]:
# Create an empty DataFrame to store the recommendations
recommendation_scores = pd.DataFrame(columns=["movie_id", "user_id", "RecommendationScore"])
recommendation_scores

,movie_id,user_id,RecommendationScore


In [7]:
# Get recommendations for every user
for user in users:
    # Get user's ratings
    user_ratings = normalized_ratings.loc[user]

    # Generate the recommendations
    rec = utils.generate_recommendation(user_ratings, W, vb, hb)

    # Construct user DataFrame
    new_recommendation = pd.DataFrame({"movie_id": normalized_ratings.columns, "user_id": user})
    new_recommendation = new_recommendation.assign(RecommendationScore = rec[0].numpy())

    # Add recommendation to DataFrame
    recommendation_scores = pd.concat([recommendation_scores, new_recommendation])

recommendation_scores

,movie_id,user_id,RecommendationScore
0,1,1,0.520410
1,2,1,0.825916
2,3,1,0.470805
3,4,1,0.712805
4,5,1,0.576460
...,...,...,...
2066,2067,1508,0.004352
2067,2068,1508,0.004824
2068,2069,1508,0.004187
2069,2070,1508,0.005847


In [8]:
# Merge all recommendation scores with original dataset ratings
# Inner merge because we cannot perform calculations on NaN values to evaluate the model
merged_df = recommendation_scores.merge(df, on=['movie_id', 'user_id'], how='inner')
merged_df

,movie_id,user_id,RecommendationScore,rating
0,1,1,0.520410,2.0
1,2,1,0.825916,4.0
2,3,1,0.470805,3.5
3,4,1,0.712805,3.0
4,5,1,0.576460,4.0
...,...,...,...,...
35489,669,1508,0.003586,1.0
35490,686,1508,0.004182,2.5
35491,693,1508,0.021464,3.5
35492,751,1508,0.003404,1.0


In [9]:
# normalize rating column
merged_df["rating"] = merged_df["rating"] / 5
merged_df

,movie_id,user_id,RecommendationScore,rating
0,1,1,0.520410,0.4
1,2,1,0.825916,0.8
2,3,1,0.470805,0.7
3,4,1,0.712805,0.6
4,5,1,0.576460,0.8
...,...,...,...,...
35489,669,1508,0.003586,0.2
35490,686,1508,0.004182,0.5
35491,693,1508,0.021464,0.7
35492,751,1508,0.003404,0.2


In [10]:
# Calculate the RMSE
from sklearn.metrics import mean_squared_error

In [11]:
rmse = mean_squared_error(merged_df["rating"], merged_df["RecommendationScore"], squared=False)
print(rmse)

0.34476740734975386


/Users/stevenhill/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
